In [2]:
## SVR Implmentation
import seaborn as sns 
df = sns.load_dataset('tips')
df.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 244 entries, 0 to 243
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   total_bill  244 non-null    float64 
 1   tip         244 non-null    float64 
 2   sex         244 non-null    category
 3   smoker      244 non-null    category
 4   day         244 non-null    category
 5   time        244 non-null    category
 6   size        244 non-null    int64   
dtypes: category(4), float64(2), int64(1)
memory usage: 7.4 KB


In [4]:
df.sex.value_counts()

sex
Male      157
Female     87
Name: count, dtype: int64

In [5]:
df.smoker.value_counts()

smoker
No     151
Yes     93
Name: count, dtype: int64

In [6]:
df.time.value_counts()

time
Dinner    176
Lunch      68
Name: count, dtype: int64

In [7]:
df.day.value_counts()

day
Sat     87
Sun     76
Thur    62
Fri     19
Name: count, dtype: int64

In [8]:
df.columns

Index(['total_bill', 'tip', 'sex', 'smoker', 'day', 'time', 'size'], dtype='object')

In [22]:
## feature encoding (lable encoding and one hot encoding)
X = df[['tip', 'sex', 'smoker', 'day', 'time', 'size']]
Y = df['total_bill']

In [23]:
## train test split
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X,Y,test_size=0.25,random_state=42)

In [24]:
X_train.head()

,tip,sex,smoker,day,time,size
115,3.50,Female,No,Sun,Dinner,2
181,5.65,Male,Yes,Sun,Dinner,2
225,2.50,Female,Yes,Fri,Lunch,2
68,2.01,Male,No,Sat,Dinner,2
104,4.08,Female,No,Sat,Dinner,2


In [25]:
## feature encoding
from sklearn.preprocessing import LabelEncoder
## for binary features
le1 = LabelEncoder()
le2 = LabelEncoder()
le3 = LabelEncoder()

In [26]:
import warnings
warnings.filterwarnings('ignore')
X_train['sex'] = le1.fit_transform(X_train['sex'])
X_train['smoker'] = le2.fit_transform(X_train['smoker'])
X_train['time'] = le3.fit_transform(X_train['time'])

In [27]:
X_train.head()

,tip,sex,smoker,day,time,size
115,3.50,0,0,Sun,0,2
181,5.65,1,1,Sun,0,2
225,2.50,0,1,Fri,1,2
68,2.01,1,0,Sat,0,2
104,4.08,0,0,Sat,0,2


In [28]:
X_test['sex'] = le1.transform(X_test['sex'])
X_test['smoker'] = le2.transform(X_test['smoker'])
X_test['time'] = le3.transform(X_test['time'])

In [29]:
X_test.head()

,tip,sex,smoker,day,time,size
24,3.18,1,0,Sat,0,2
6,2.00,1,0,Sun,0,2
153,2.00,1,0,Sun,0,4
211,5.16,1,1,Sat,0,4
198,2.00,0,1,Thur,1,2


In [36]:
## One Hot Encoding For day column

from sklearn.compose import ColumnTransformer

from sklearn.preprocessing import OneHotEncoder

ct=ColumnTransformer(transformers=[('onehot',OneHotEncoder(drop='first'),[3])],remainder='passthrough')

In [37]:
ct.fit_transform(X_train)

array([[0., 1., 0., ..., 0., 0., 2.],
       [0., 1., 0., ..., 1., 0., 2.],
       [0., 0., 0., ..., 1., 1., 2.],
       ...,
       [0., 0., 0., ..., 1., 0., 2.],
       [0., 1., 0., ..., 1., 0., 2.],
       [1., 0., 0., ..., 1., 0., 3.]], shape=(183, 8))

In [39]:
import sys
import numpy
numpy.set_printoptions(threshold=sys.maxsize)
X_train = ct.fit_transform(X_train)
X_test = ct.transform(X_test)

In [42]:
## Apply SVR
from sklearn.svm import SVR
from sklearn.metrics import r2_score, mean_absolute_error
svr = SVR()
svr.fit(X_train,Y_train)
Y_pred = svr.predict(X_test)
print('R2 Square : ',r2_score(Y_test,Y_pred))
print('MAE : ',mean_absolute_error(Y_test,Y_pred))

R2 Square :  0.49798620106004743
MAE :  4.463296539661222


In [43]:
## Apply hyper parameter tuning 
from sklearn.model_selection import GridSearchCV

## defining parameter range for hyperparameters
param_grid = {'C': [0.1, 1, 10, 100, 1000], 
              'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
              'kernel': ['rbf']} 


In [44]:
grid = GridSearchCV(SVR(),param_grid,refit=True,verbose=3)

In [45]:
grid.fit(X_train,Y_train)

Fitting 5 folds for each of 25 candidates, totalling 125 fits
[CV 1/5] END .......C=0.1, gamma=1, kernel=rbf;, score=-0.114 total time=   0.1s
[CV 2/5] END .......C=0.1, gamma=1, kernel=rbf;, score=-0.008 total time=   0.0s
[CV 3/5] END .......C=0.1, gamma=1, kernel=rbf;, score=-0.003 total time=   0.0s
[CV 4/5] END .......C=0.1, gamma=1, kernel=rbf;, score=-0.015 total time=   0.0s
[CV 5/5] END .......C=0.1, gamma=1, kernel=rbf;, score=-0.123 total time=   0.0s
[CV 1/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.019 total time=   0.0s
[CV 2/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.096 total time=   0.0s
[CV 3/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.078 total time=   0.0s
[CV 4/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.099 total time=   0.0s
[CV 5/5] END .....C=0.1, gamma=0.1, kernel=rbf;, score=-0.020 total time=   0.0s
[CV 1/5] END ....C=0.1, gamma=0.01, kernel=rbf;, score=-0.102 total time=   0.0s
[CV 2/5] END .....C=0.1, gamma=0.01, kernel=rbf

GridSearchCV(estimator=SVR(),
             param_grid={'C': [0.1, 1, 10, 100, 1000],
                         'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
                         'kernel': ['rbf']},
             verbose=3)

In [46]:
Y_pred_ = grid.predict(X_test)

In [47]:
print(mean_absolute_error(Y_test,Y_pred_))
print(r2_score(Y_test,Y_pred_))

4.260495176171523
0.5630706781167809
